# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [ ]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.io.cli import CLIInterface
import pandas as pd
from veg2hab.bronnen import FGR, Bodemkaart, LBK

pd.set_option('display.max_columns', 100)

CLIInterface.get_instance()

### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [ ]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [ ]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

## Inladen van een kartering vanaf een shapefile

In [ ]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
Kartering.from_shapefile(shp_path, vegtype_col_format="single", sbb_of_vvn="SBB", ElmID_col="elmid", SBB_col=["SBBTYPE"], VvN_col=[], split_char="+", lok_vegtypen_col=["Vegtype"]).gdf

## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [ ]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


### Inladen bronnen

In [ ]:
mask = access_kartering.get_geometry_mask()
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
bodemkaart = Bodemkaart.from_file(Path("../data/bronbestanden/bodemkaart.gpkg"), mask=mask)
lbk = LBK.from_file(Path("../data/bronbestanden/lbk.shp"), mask=mask)

#### Opzoeken en toevoegen VvN aan kartering

In [ ]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [ ]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

In [ ]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

### De mitsen en mozaiekregels checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [ ]:
# access_kartering.bepaal_habitatkeuzes(fgr, bodemkaart, lbk)
access_kartering.bepaal_mits_habitatkeuzes(fgr, bodemkaart, lbk)
access_kartering.bepaal_mozaiek_habitatkeuzes()

In [ ]:
access_kartering.gdf.head()

### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [ ]:
final_format = access_kartering.as_final_format()
final_format

In [ ]:
final_format

In [ ]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))